In [109]:
# Imports

import os, sys
import random
import time
import numpy as np

from natsort import natsorted
from collections import Counter

import pickle
from model import CNN
import torch

import PIL
from PIL import Image
from PIL import ImageDraw as draw
from PIL import ImageFont as ifont
import ImageGenerator as IG

from IPython.display import clear_output


class Board:
#     strView = ''
    def __init__(self):
        self.myBoard = {}
        for i in range(24):
            self.myBoard[i] = 0
        self.myBoard[0] = 2
        self.myBoard[5] = -5
        self.myBoard[7] = -3
        self.myBoard[11] = 5
        self.myBoard[12] = -5
        self.myBoard[16] = 3
        self.myBoard[18] = 5
        self.myBoard[23] = -2
        self.maxRows = 5
        self.xFree = 0
        self.oFree = 0
        self.xJail = 0
        self.oJail = 0
        self.xHome = 5
        self.oHome = 5
        
        self.alfabet = IG.load_alfabet()
        self.alfabet_images = {}
        for a in self.alfabet:
            self.alfabet_images[a] = Image.open(self.alfabet[a][1], 'r')
        
        self.mapping_side = {
            False: -1,
            True: 1
        }
        
        self.strView = ''        
        
    def toImage(self, step_i, folder='.', save=False):
        lines = self.__repr__().split('\n')
        max_count = 0
        lines = lines[2:-3]

        for l in lines:
            max_count = max(len(l), max_count)
        w, h = IG.get_image_size(max_count, len(lines))
        img = Image.new("L", (w, h), 255)
        w_step, h_step = IG.get_subimage_step()
        h_i = 0
        for l in lines:
            w_i = 0
            fill_l = l if len(l) == max_count else l + " " * (max_count - len(l))
            for s in fill_l:
                img.paste(self.alfabet_images[s], (w_i, h_i))
                w_i += w_step
            h_i += h_step
            
        if save:
            img.save(os.path.join(folder, str(step_i) + ".png"))        
        return img
    
    def filter_best_moves(self, availableMoves):
        if len(availableMoves) > 0:
            max_score = np.max([i[3] for i in availableMoves])
            return [i for i in availableMoves if i[3]>=max_score ]
        else:
            return []
    
    def getPossibleMoves(self, side, allSteps=[]):
        
        allSpaces = list(range(len(self.myBoard)))
        if (side and self.xJail>0) or (not side and self.oJail>0):
            allSpaces.append(-1)    
            
        availableMoves = []
        for steps in allSteps:
            for space in allSpaces:
                if side:                    
                    if self.xJail > 0 and space != -1:                     
                        continue
                    elif self.xJail > 0 and space == -1:
                        if self.myBoard[24-steps] > 1:
                            continue
                        elif self.myBoard[24-steps] == 1:
                            availableMoves.append([space, side, steps, 0.5])
                        else:
                            availableMoves.append([space, side, steps, 0.0])
                    elif space not in self.myBoard:
                        continue
                    elif self.myBoard[space] >= 0:
                        continue
                    else:
                        newSpace = space - steps
                        if newSpace < 0:
                            if self.xHome < 15:
                                continue
                            else:
                                availableMoves.append([space, side, steps, 1.0])
                        elif (self.myBoard[newSpace] > 1):
                            continue
                        elif (self.myBoard[newSpace] == 1):
                            availableMoves.append([space, side, steps, 0.5])
                        else:         
                            availableMoves.append([space, side, steps, 0.1])
        
                else:
                    if self.oJail > 0 and space != -1:
                        continue                    
                    elif self.oJail > 0 and space == -1:
                        if -self.myBoard[steps-1] > 1 :
                            continue
                        elif -self.myBoard[steps-1] == 1:                        
                            availableMoves.append([space, side, steps, 0.5])                            
                        else:
                            availableMoves.append([space, side, steps, 0.0])
                    elif space not in self.myBoard:
                        continue
                    elif -self.myBoard[space] >= 0:
                        continue
                    else:                        
                        newSpace = space + steps
                        if newSpace > 23:
                            if self.oHome < 15:
                                continue
                            else:
                                availableMoves.append([space, side, steps, 1.0])
                        elif -self.myBoard[newSpace] > 1:
                            continue
                        elif -self.myBoard[newSpace] == 1:
                            availableMoves.append([space, side, steps, 0.5])
                        else:
                            availableMoves.append([space, side, steps, 0.1])
                            
        return natsorted(availableMoves)
            

    def makeMove(self, space, side, steps):
        """
        @param
        space = starting space
        side = true if X (steps are decreasing), false if O (steps are increasing)
        steps = number of steps
        *** if you are trying to free a piece from jail, steps = 0
        @return
        tuple of true if move was made, false if not possible, and string with response
        """
        if side:
            
            if self.xJail > 0 and space != -1: 
                return (False, "Make sure you free your piece from jail first!")
            elif self.xJail > 0 and space == -1:
                newSpace = 24 - steps
                if self.myBoard[newSpace] > 1:
                    return (False, "Space is occupied")
                elif self.myBoard[newSpace] == 1:
                    self.xJail -= 1
                    self.myBoard[newSpace] = -1
                    self.oJail += 1
                    self.oHome -= 1
                    return (True, "Piece released from jail and sent one to jail!")
                else:
                    self.xJail -= 1
                    self.myBoard[newSpace] -= 1
                    return (True, "Piece released from jail!")
            elif space not in self.myBoard:
                return (False, "Space is wrong")
            elif (self.myBoard[space] >= 0):
                return (False, "Space is either empty or wrong team")
            else:
                newSpace = space - steps
                if newSpace < 0:
                    if self.xHome < 15:
                        return (False, "Make sure all of your pieces are in home before clearing them!")
                    else:
                        self.myBoard[space] += 1
                        self.xFree += 1
                        return (True, "Piece cleared!")
                elif (self.myBoard[newSpace] > 1):
                    return (False, "New space is occupied")
                elif (self.myBoard[newSpace] == 1):
                    self.myBoard[space] += 1
                    self.myBoard[newSpace] = -1
                    self.oJail += 1
                    if newSpace >= 18:
                        self.oHome -= 1
                    return (True, "Sent one to jail!")
                else:
                    self.myBoard[space] += 1
                    self.myBoard[newSpace] -= 1
                    if newSpace < 6:
                        self.xHome += 1
                    if newSpace < 11:
                        self.updateRows(True)
                    else:
                        self.updateRows(False)
                    return (True, "Move made")
        else:
            if self.oJail > 0 and space != -1:
                return (False, "Make sure you free your piece from jail first!")
            elif self.oJail > 0 and space == -1:
                newSpace = -1+steps
                if -self.myBoard[newSpace] > 1 :
                    return (False, "Space is occupied")
                elif -self.myBoard[newSpace] == 1: 
                    self.oJail -= 1
                    self.myBoard[newSpace] = 1
                    self.xJail += 1
                    self.xHome -= 1
                    return (True, "Piece released from jail and one sent to jail!")
                else:
                    self.oJail -= 1
                    self.myBoard[newSpace] +=  1
                    return (True, "Piece released from jail!")
            elif space not in self.myBoard:                
                return (False, "Space is wrong")
            elif -self.myBoard[space] >= 0:
                return (False, "Space is either empty or wrong team")
            else:
                newSpace = space + steps
                if newSpace > 23:
                    if self.oHome < 15:
                        return (False, "Make sure all of your pieces are in home before clearing them!")
                    else:
                        self.myBoard[space] -= 1
                        self.oFree += 1
                        return (True, "Piece cleared!")
                elif -self.myBoard[newSpace] > 1:
                    return (False, "New space is occupied")
                elif -self.myBoard[newSpace] == 1:
                    self.myBoard[space] -= 1
                    self.myBoard[newSpace] = 1
                    self.xJail += 1
                    if newSpace < 6:
                        self.xHome -= 1
                    return (True, "Sent one to jail!")
                else:
                    self.myBoard[space] -= 1
                    self.myBoard[newSpace] += 1
                    if newSpace > 17:
                        self.oHome += 1
                    if newSpace < 11:
                        self.updateRows(True)
                    else:
                        self.updateRows(False)
                    return (True, "Move made")

    def updateRows(self, top):
        changed = False
        if top:
            for i in range(12):
                if (self.myBoard[i] > 5):
                    self.maxRows = self.myBoard[i]
                    changed = True
        else:
            for i in range(23,11,-1):
                if (self.myBoard[i] > 5):
                    self.maxRows = self.myBoard[i]
                    changed = True
        if not changed:
            self.maxRows = 5


    def __repr__(self):
        """
        1 -> 1
        2 -> 5
        3 -> 9
        4 -> 13
        5 -> 17
        6 -> 21
        7 -> 28
        8 -> 32
        9 -> 36
        10 -> 40
        11 -> 44
        12 -> 48
        13 -> 1
        17 -> 17
        19 -> 28
        24 -> 48
        """

        emptyline = "|                       | |                       |"
#         if (self.oJail > 0):
        boardstring = "	                O Jail: " + str(self.oJail) + "\n"
        boardstring += "                X HOME BOARD     Freed:" + str(self.xFree) + "\n"
#         else:
#             boardstring = "	            X HOME BOARD     Freed:" + str(self.xFree) + "\n"
        boardstring += " -------------------------------------------------\n"
        boardstring += "|12  11  10  9   8   7  | | 6   5   4   3   2   1 |\n"
        for i in range(self.maxRows):
            boardstring += self.populateTop(i)
        boardstring += " ------------------------------------------------- \n"
        for i in range(self.maxRows-1,-1,-1):
            boardstring += self.populateBottom(i)
        boardstring += "|13  14  15  16  17  18 | | 19  20  21  22  23  24|\n"
        boardstring += " -------------------------------------------------\n"
        boardstring += "                O HOME BOARD     Freed: " + str(self.oFree) + "\n"
#         if (self.xJail > 0):
        boardstring += "                    X Jail: " + str(self.xJail) + "\n"
        self.strView = boardstring
        return boardstring


    def populateTop(self, lineNumber):
        """
        1 -> 1
        2 -> 5
        3 -> 9
        4 -> 13
        5 -> 17
        6 -> 21
        7 -> 28
        8 -> 32
        9 -> 36
        10 -> 40
        11 -> 44
        12 -> 48
        """
        line = "|                       | |                       |\n"
        boardtostring = {} 
        boardtostring[0] = 48
        boardtostring[1] = 44
        boardtostring[2] = 40
        boardtostring[3] = 36
        boardtostring[4] = 32
        boardtostring[5] = 28
        boardtostring[6] = 21
        boardtostring[7] = 17
        boardtostring[8] = 13
        boardtostring[9] = 9
        boardtostring[10] = 5
        boardtostring[11] = 1
        for i in range(12):
            if (self.myBoard[i] > lineNumber):
                line = line[:boardtostring[i]] + 'O' + line[boardtostring[i]+1:]
            elif (abs(self.myBoard[i]) > lineNumber and self.myBoard[i] < 0):
                line = line[:boardtostring[i]] + 'X' + line[boardtostring[i]+1:]
        return line

    def populateBottom(self, lineNumber):
        line = "|                       | |                       |\n"
        boardtostring = {}
        boardtostring[11] = 48
        boardtostring[10] = 44
        boardtostring[9] = 40
        boardtostring[8] = 36
        boardtostring[7] = 32
        boardtostring[6] = 28
        boardtostring[5] = 21
        boardtostring[4] = 17
        boardtostring[3] = 13
        boardtostring[2] = 9
        boardtostring[1] = 5
        boardtostring[0] = 1
        for i in range(12):
            if (self.myBoard[12+i] > lineNumber):
                line = line[:boardtostring[i]] + 'O' + line[boardtostring[i]+1:]
            elif (abs(self.myBoard[12+i]) > lineNumber and self.myBoard[12+i] < 0):
                line = line[:boardtostring[i]] + 'X' + line[boardtostring[i]+1:]
        return line

In [129]:
# Load models

MODEL_CNN_PATH = os.path.join('models','model_n0.pt')
MODEL_QLEARN_PATH = os.path.join('models','model_q1.pkl')

IG.create_alfabet()
cnn = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn.load_state_dict(torch.load(MODEL_CNN_PATH))
if device != "cpu":
    cnn.cuda(0)    
    
if os.path.exists(MODEL_QLEARN_PATH):
    with open(MODEL_QLEARN_PATH, 'rb') as f:
        qlearn = pickle.load(f)
else:
    qlearn = {}
    
    
# Game functions
    
exitTerms = ("quit", "exit", "bye","q")

def main(gameNumber=0, VERBOSE = True, SAVE_IMG = False, GAME_TYPE=None, QLEARN = False):    
    
    GAME_TYPES = ['pvp', 'pvr', 'pvq', 'pvd', 'pvc', 'rvr', 'qvr', 'dvr', 'cvr']
    PLAYER_TYPES = {
        'pvp':{True: 'person', False:'person'}, 
        'pvr':{True: 'person', False:'random'}, 
        'pvq':{True: 'person', False:'qlearn'}, 
        'pvd':{True: 'person', False:'dlearn'},
        'pvc':{True: 'person', False:'cnn'},
        'rvr':{True: 'random', False:'random'}, 
        'qvr':{True: 'qlearn', False:'random'},
        'dvr':{True: 'dlearn', False:'random'},
        'cvr':{True: 'cnn', False:'random'}
    }
    
    b = Board()   
    
    if VERBOSE:    
        with open('readme.txt', 'r') as intro:  
            for line in intro:        
                print(line.replace('\n',''))                
                
    if GAME_TYPE == None:        
        print('What do you want to play? Available game types: {}'.format(GAME_TYPES))
        GAME_TYPE = input() 
        
    line = GAME_TYPE
        
    if GAME_TYPE not in GAME_TYPES:
        print('This game type is not available')
        
    else:
        player_X = PLAYER_TYPES[GAME_TYPE][True]
        player_O = PLAYER_TYPES[GAME_TYPE][False]
        
        iMove = 0        
        SIDE = True #True if X, false if O
        
        while (line not in exitTerms and (b.xFree < 15 or b.oFree < 15)):            
            
            if VERBOSE:
                clear_output()
                print(b)    
            
            roll1 = random.randint(1,6)
            roll2 = random.randint(1,6)
            turnComplete = False
            
            total = roll1 + roll2
            if roll1 == roll2:
                total *= 2
                
            rolls = [roll1, roll2] if roll1 != roll2 else [roll1, roll2]*2            
            availableMoves = b.getPossibleMoves(SIDE, rolls) 
            
            if VERBOSE:
                if SIDE:                
                    print("X, You rolled dice and you have " + '+'.join([str(r) for r in rolls]) + ' steps')
                else:
                    print("O, You rolled dice and you have " + '+'.join([str(r) for r in rolls]) + ' steps')
                
            while (not turnComplete and line not in exitTerms and len(rolls) > 0):                
                
                if VERBOSE:
                    print("Possible Moves:" , [str(m[0]+1)+', '+ str(m[2]) for m in availableMoves])
                
                if SIDE:
                    if player_X in ['random', 'qlearn', 'dlearn']:
                        if player_X == 'qlearn':
                            availableMoves = b.filter_best_moves(availableMoves)                            
                        elif player_X == 'dlearn':
                            
                            hist_state_str = str(list(b.myBoard.values()) + [b.oJail, b.xJail])                            

                            if hist_state_str in qlearn:
                                if SIDE in qlearn[hist_state_str]:
                                    
                                    for iam,availMove in enumerate(availableMoves):
                                        availableMoves[iam][3] = 0.0
                                        if availMove[2] in qlearn[hist_state_str][SIDE]:
                                            if availMove[0] in qlearn[hist_state_str][SIDE][availMove[2]]:
                                                score = qlearn[hist_state_str][SIDE][availMove[2]][availMove[0]] 
                                                availableMoves[iam][3] = score                                            
                            
                            availableMoves = b.filter_best_moves(availableMoves)                        
                        if len(availableMoves) > 0:
                            chosenMove = random.sample(availableMoves, 1)[0]
                            line = str(chosenMove[0]+1)+', '+ str(chosenMove[2])
                        else:
                            line = 'f'
                    elif player_X == 'cnn':                        
                        if len(availableMoves) > 0:
                            _steps = [[m[0],m[2]] for m in availableMoves]

                            np_img = np.ndarray(shape=(1,1,256,128))
                            np_img[0,0] = np.array( b.toImage(step_i=0, save=False).resize((256,128))).transpose((1,0))
                            _image = torch.tensor(np_img).float().to(device)

                            res_tesnors = []
                            for step in _steps:
                                step = torch.tensor([step]).float().to(device)
                                res_tesnors.append( cnn(_image, step) )

                            availableMoves = [[m[0],m[1],m[2],1-score.item()] for m, score in zip(availableMoves, res_tesnors)]
                            availableMoves = b.filter_best_moves(availableMoves)    
                            
                            chosenMove = random.sample(availableMoves, 1)[0]
                            line = str(chosenMove[0]+1)+', '+ str(chosenMove[2])                            
                        else:
                            line = 'f'
                    else:
                        line = input()                        
                else:
                    if player_O in ['random', 'qlearn', 'dlearn']:
                        if player_O in ['qlearn', 'dlearn']:
                            availableMoves = b.filter_best_moves(availableMoves)                        
                        if len(availableMoves)>0:
                            chosenMove = random.sample(availableMoves, 1)[0]
                            line = str(chosenMove[0]+1)+', '+ str(chosenMove[2])
                        else:
                            line = 'f'
                            
                    elif player_O == 'cnn':                        
                        if len(availableMoves) > 0:
                            _steps = [[m[0],m[2]] for m in availableMoves]

                            np_img = np.ndarray(shape=(1,1,256,128))
                            np_img[0,0] = np.array( b.toImage(step_i=0, save=False).resize((256,128))).transpose((1,0))
                            _image = torch.tensor(np_img).float().to(device)

                            res_tesnors = []
                            for step in _steps:
                                step = torch.tensor([step]).float().to(device)
                                res_tesnors.append( cnn(_image, step) )

                            availableMoves = [[m[0],m[1],m[2],score.item()] for m, score in zip(availableMoves, res_tesnors)]
                            availableMoves = b.filter_best_moves(availableMoves)    
                            
                            chosenMove = random.sample(availableMoves, 1)[0]
                            line = str(chosenMove[0]+1)+', '+ str(chosenMove[2])                            
                        else:
                            line = 'f'
                    else:
                        line = input()
                        
                if VERBOSE:
                    print('Сhoise %s' % line)
                else:
                    print('{}: {}'.format(SIDE, line))
                
                try:
                    space,steps = parseInput(line)
                except:
                    if VERBOSE:
                        print('Wrong Input')
                    continue
                
                # Game logic andrules compliance check
                jailFreed, jailCase = False, False
                
                if (SIDE and b.xJail > 0) or (not SIDE and b.oJail > 0):
                    jailCase = True
                    
                if space == 100 and steps == 100:
                    total = 0
                    rolls = []
                    break
                if space == 101 and steps == 101:
                    break
                if steps not in rolls and steps != 100 and not jailCase:
                    if VERBOSE:
                        print("You didn't roll that!")
                    continue
                    
                if not jailCase and space==0:
                    if VERBOSE:
                        print("No checkers in the jail!")             
                
                if space == 0 and SIDE and b.xJail > 0:
                    jailFreed = True
                elif space == 0 and not SIDE and b.oJail > 0:
                    jailFreed = True
                        
                space = space - 1
                if space < -1 or space > 23:# or steps < 0:
                    if VERBOSE:
                        print("That move is not allowed.  Please try again.")
                    continue
                    
                # History 
                # board_state = copy.deepcopy(b.my) 
                hist_state = list(b.myBoard.values()) + [b.oJail, b.xJail]
                hist_move = [space, SIDE, steps]
                move, response = b.makeMove(space, SIDE, steps)
                if VERBOSE:
                    print(response)
                
#                 if move and jailFreed:
#                     steps = tempSteps
                    
                if move:
                    total = total - steps
                    if VERBOSE:
                        clear_output()
                        print(b)
                                    
                    MOVES_HISTORY.append({'hist_state':hist_state, 'hist_move':hist_move, 'available_moves':availableMoves})
                        
                    
                    if SAVE_IMG:
                        b.toImage(iMove, CURRENT_FOLDER, save=True)
                    iMove+=1  
                    
#                     if steps in rolls:
                    rolls.remove(steps)
                    if VERBOSE:
                        print("You have " + str(total) + ' (' + '+'.join([str(r) for r in rolls]) + ") steps left.")  
                    
                availableMoves = b.getPossibleMoves(SIDE, rolls)
            
            if b.oFree>=15:
                print('O wins!')
                wins.append(['game'+str(gameNumber), SIDE])
                break
            elif b.xFree>=15:
                print('X wins!')
                wins.append(['game'+str(gameNumber), SIDE])
                break
                
            if len( b.getPossibleMoves(True, [1,2,3,4,5,6]) )==0 and len( b.getPossibleMoves(False, [1,2,3,4,5,6]) )==0:
                if VERBOSE:
                    print('No available moves!')
                break
                
            SIDE = not SIDE    


#TODO: Include error management
def parseInput(response):
    if response == "d" or response == "f" or response == "done" or response == "finish":
        return(100,100)
    if response in exitTerms:
        return (101, 101)
    # if type(response) == type("Sample string"):
    # 	return(101,101)
    loc = findSeparation(response)
    return(int(response[:loc]), int(response[loc+1:])) 

def findSeparation(value):
    for i in range(len(value)):
        if (value[i] == ' ' or value[i] == ','):
            return i
    return 0

In [119]:
wins = []
MOVES_HISTORY = [] 
WINNER = None

if __name__ == "__main__":
    main(gameNumber=0, VERBOSE = False, SAVE_IMG = False, GAME_TYPE='cvr')

True: 24, 2
True: 24, 2
True: 22, 2
True: 22, 2
False: 17, 1
False: 1, 1
False: 1, 1
False: 18, 1
True: 13, 2
True: 8, 5
False: 12, 5
False: 12, 5
False: 19, 5
False: 17, 5
True: 13, 3
True: 13, 5
False: 12, 5
False: 19, 5
False: 2, 5
False: 7, 5
True: 6, 1
True: 5, 4
False: 12, 5
False: 19, 3
True: 6, 5
True: 8, 6
False: 0, 5
False: 17, 5
False: 19, 5
False: 17, 5
True: 2, 1
True: 11, 3
False: 5, 6
False: 19, 3
True: 13, 2
True: 13, 4
False: 0, 4
False: 4, 1
True: 9, 5
True: 8, 4
False: 5, 5
False: 10, 1
True: 0, 6
True: 0, 4
False: 0, 3
False: 3, 4
True: 0, 4
True: 21, 2
False: 11, 3
False: 14, 3
False: 12, 3
False: 7, 3
True: 21, 2
True: 6, 3
False: 17, 5
False: 10, 3
True: 20, 4
True: 20, 5
False: 0, 5
False: 12, 6
True: 19, 3
True: 19, 4
False: 22, 2
False: 18, 1
True: 0, 2
True: 6, 1
False: 0, 2
False: 24, 1
True: 23, 2
True: 21, 6
False: 22, 3
False: 19, 5
True: 5, 4
True: 8, 6
False: 0, 2
False: 17, 2
False: 19, 2
False: 22, 2
True: 0, 6
True: 4, 3
False: 17, 5
False: 22, 6
Tru

In [127]:
# Launch N games for Q Learning process

def qlearn_batch():
    learning_rate = 0.01

    WINNER = wins[len(wins)-1][1]

    for move in MOVES_HISTORY:

        hist_state = str(move['hist_state'])
        SIDE = move['hist_move'][1]
        steps = move['hist_move'][2]
        space = move['hist_move'][0]

        if hist_state not in QLEARN_MEMORY:
            QLEARN_MEMORY[hist_state] = {}
        if SIDE not in QLEARN_MEMORY[hist_state]:
            QLEARN_MEMORY[hist_state][SIDE] = {}
        if steps not in QLEARN_MEMORY[hist_state][SIDE]:
            QLEARN_MEMORY[hist_state][SIDE][steps] = {}
        if space not in QLEARN_MEMORY[hist_state][SIDE][steps]:
            QLEARN_MEMORY[hist_state][SIDE][steps][space] = 0.0    

        if SIDE == WINNER:
            QLEARN_MEMORY[hist_state][SIDE][steps][space] += learning_rate
        else:
            QLEARN_MEMORY[hist_state][SIDE][steps][space] -= learning_rate  
            
epochs = 10000    

QLEARN_MEMORY = {}
wins = []
MOVES_HISTORY = []    

for igame in range(epochs): 
    print('Game {}/{}'.format(igame+1, epochs))
    MOVES_HISTORY = []
    iMove = 0  
    main(gameNumber=igame, VERBOSE = False, SAVE_IMG = False, GAME_TYPE='rvr') 
    qlearn_batch()
    clear_output()    
    
qlearn = QLEARN_MEMORY

with open(MODEL_QLEARN_PATH, 'wb') as fp:
    pickle.dump(qlearn, fp)

In [ ]:
# Launch N games for testing

epochs = 10000

wins = []
MOVES_HISTORY = [] 

for igame in range(epochs): 
    print('Game {}/{}'.format(igame+1, epochs))
    MOVES_HISTORY = []
    iMove = 0  
    main(gameNumber=igame, VERBOSE = False, SAVE_IMG = False, GAME_TYPE='cvr')      
    clear_output()
    
Counter([wn[1] for wn in wins])    

Game 3005/10000
True: 24, 3
True: 24, 6
False: 17, 1
False: 18, 6
True: 0, 5
True: 21, 1
False: 1, 4
False: 19, 5
True: 13, 5
True: 13, 4
False: 1, 4
False: 19, 3
True: 20, 2
True: 6, 5
False: 5, 5
False: 17, 6
True: 20, 2
True: 13, 2
True: 18, 2
True: 18, 2
False: 17, 2
False: 12, 3
True: 16, 2
True: 6, 4
False: 12, 3
False: 15, 4
True: 2, 1
True: 9, 4
False: 0, 2
False: 2, 1
True: 14, 6
True: 11, 3
False: 15, 4
False: 12, 2
True: 16, 1
True: 15, 5
False: 0, 4
False: 3, 6
True: 5, 4

In [12]:
# Generate dataset of images and vectors for CNN

import os
import pandas as pd

if not os.path.exists( 'data' ):
    os.mkdir('data')

prefix = 'data/game'
wins = []
header = list(range(0,24)) + ['oJail', 'xJail'] + ['ChosenMove', 'Player' ,'ChosenTurnNumber', 'AvailableMoves']

for igame in range(200):
    
    CURRENT_FOLDER = prefix+str(igame)
    
    if not os.path.exists( CURRENT_FOLDER ):
        os.mkdir(CURRENT_FOLDER)
        
    MOVES_HISTORY = []
    iMove=0  
    
    main(igame, VERBOSE = True, SAVE_IMG = True, GAME_TYPE='rvr')    
    
    filename = CURRENT_FOLDER + '/moves.csv'
    df = pd.DataFrame( [i['hist_state']+i['hist_move']+[i['available_moves']] for i in MOVES_HISTORY] , columns = header )
    df.to_csv(filename, index=False)
    
    if igame>2:
        break
        
df_wins=pd.DataFrame( wins )
df_wins.columns = ['game', 'winner']
df_wins.to_csv('data/result.txt',index=False) 